In [1]:
# 1. Ответы на вопросы

### 1.1 Найдите аналитическое решение задачи о регрессии. Используйте векторную форму уравнения.

  Векторная форма:

  - $y=Xw+e$
    - $y$ - вектор наблюдаемых значений зависимой переменной (размерности $n×1$);
    - $X$ - матрица признаков (размерности $n×m$), где $n$ — количество наблюдений, а $m$ — количество признаков;
    - $w$ - вектор коэффициентов регрессии (размерности $m×1$);
    - $e$ - вектор ошибок (размерности $n×1$);

  _Цель состоит в нахождении вектора коэффициентов $w$, который минимизирует сумму квадратов ошибок (метод наименьших квадратов, МНК)._

  Функция потерь для метода наименьших квадратов записывается как:

  - $L(w)=∥y−Xw∥^2=(y−Xw)^⊤(y−Xw)$
  
  _Для минимизации этой функции необходимо найти $w$, при котором $L(w)$ достигает минимума._

  Вектор коэффициентов $w$, который минимизирует сумму квадратов ошибок, вычисляется по формуле:

  - $w=(X^TX)^{−1}X^Ty$
    - $X^⊤X$: Это матрица, которая учитывает взаимосвязи между признаками (столбцами матрицы $X$);
    - $X^⊤y$: Это вектор, который показывает, как признаки $X$ связаны с целевой переменной $y$;
    - $(X^⊤X)^−1$: Это обратная матрица, которая "убирает" влияние корреляции между признаками, чтобы найти оптимальные коэффициенты $w$.

### 1.2 Что меняется в решении при добавлении регуляризации L1 и L2 к функции потерь.

  **L2** - Гребневая регрессия

  Добавляет к линейной регрессии дополнительное смещение в виде штрафа, отчего та хуже подгоняется к данным.

  Функция потерь с $L_2$:

  - $L(w)=||y - Xw||^2 + λ||w||^2_2$

  Аналитическое решение:
  
  - $w = (X^T X + λI)^{-1} X^T y$
    - $λ$ - коэффициент регуляризации L2;
    - $I$ - единичная матрица;

  **Решение становится более устойчивым, коэффициенты $w$ уменьшаются, но все признаки остаются в модели.**

  **L1** - Лассо-регрессия
  
  Лассо-регрессия стремится полностью исключить веса наименее важных признаков (т.е. устанавливая их в 0) и выдает разреженную модель с несколькими ненулевыми весами объектов.
  
  Функция потерь с $L_1$:

  - $L(w)=||y - Xw||^2 + λ||w||_1$

  Аналитическое решение:

  - Становится недоступным из за недифференцируемости функции в точках, где $w_i=0$.

**Некоторые коэффициенты $w$ становятся равными нулю, что упрощает модель и делает её разреженной**


### 1.3 Объясните, почему регуляризация L1 обычно используется для выбора объектов. Почему существует множество весов, которые равны 0 после подгонки модели?

1. **Почему регуляризация L1 обычно используется для выбора объектов?**

    -  $L_1$​ регуляризация добавляет штраф, пропорциональный сумме абсолютных значений весов (коэффициентов) модели. Приводит к уменьшению весов до нуля (исключение признаков).
  
2. **Почему существует множество весов, которые равны 0 после подгонки модели?**
   
   - **Свойство абсолютной величины**;
   - **Интерпретируемость** (выбирает один из корреляирующий признаков, остальные приводит к 0);
   - **Разреженность модели** (создает модели, где многие веса равны нулю, когда данные содержат много нерелевантных или избыточных признаков);

### 1.4 Объясните, как вы можете использовать одни и те же модели (линейная регрессия, гребневая и т.д.), но сделать возможным подгонку под нелинейные зависимости?

Линейные модели могут быть адаптированы для подгонки под нелинейные зависимости с помощью 
- преобразования признаков; 
- добавления взаимодействий; 
- применения регуляризации; 
- комбинирование моделей;


# 2. Введение 

### 2.1 Импорт библиотек.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score

from collections import Counter

### 2.2 Прочитайте обучающие и тестовые части.

In [3]:
df_train = pd.read_json('data/train.json.zip', compression='zip')
df_test = pd.read_json('data/test.json.zip', compression='zip')

In [4]:
df_train.head(2)

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level
4,1.0,1,8579a0b0d54db803821a35a4a615e97a,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,"[Dining Room, Pre-War, Laundry in Building, Di...",40.7108,7170325,-73.9539,a10db4590843d78c784171a107bdacb4,[https://photos.renthop.com/2/7170325_3bb5ac84...,2400,145 Borinquen Place,medium
6,1.0,2,b8e75fc949a6cd8225b455648a951712,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7513,7092344,-73.9722,955db33477af4f40004820b4aed804a0,[https://photos.renthop.com/2/7092344_7663c19a...,3800,230 East 44th,low


In [5]:
df_test.head(2)

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address
0,1.0,1,79780be1514f645d7e6be99a3de696c5,2016-06-11 05:29:41,Large with awesome terrace--accessible via bed...,Suffolk Street,"[Elevator, Laundry in Building, Laundry in Uni...",40.7185,7142618,-73.9865,b1b1852c416d78d7765d746cb1b8921f,[https://photos.renthop.com/2/7142618_1c45a2c8...,2950,99 Suffolk Street
1,1.0,2,0,2016-06-24 06:36:34,Prime Soho - between Bleecker and Houston - Ne...,Thompson Street,"[Pre-War, Dogs Allowed, Cats Allowed]",40.7278,7210040,-74.0000,d0b5648017832b2427eeb9956d966a14,[https://photos.renthop.com/2/7210040_d824cc71...,2850,176 Thompson Street


### 2.3 Функция предварительной обработки 'interest_level'.

In [6]:
df_copy = df_train.copy() 

In [7]:
df_copy['interest_level'] = df_copy['interest_level'].replace({'low': 0, 'medium': 1, 'high': 2}) # Заменяем значения
df_copy['interest_level']

/tmp/ipykernel_19957/2233018344.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_copy['interest_level'] = df_copy['interest_level'].replace({'low': 0, 'medium': 1, 'high': 2}) # Заменяем значения


4         1
6         0
9         1
10        1
15        0
         ..
124000    0
124002    1
124004    1
124008    1
124009    2
Name: interest_level, Length: 49352, dtype: int64

# 3. Вводный анализ данных

### 3.1 Давайте создадим дополнительные функции для повышения качества модели. Рассмотрим столбец под названием "features". Он состоит из списка основных характеристик текущей квартиры.

In [8]:
df_copy['features']

4         [Dining Room, Pre-War, Laundry in Building, Di...
6         [Doorman, Elevator, Laundry in Building, Dishw...
9         [Doorman, Elevator, Laundry in Building, Laund...
10                                                       []
15        [Doorman, Elevator, Fitness Center, Laundry in...
                                ...                        
124000              [Elevator, Dishwasher, Hardwood Floors]
124002    [Common Outdoor Space, Cats Allowed, Dogs Allo...
124004    [Dining Room, Elevator, Pre-War, Laundry in Bu...
124008    [Pre-War, Laundry in Unit, Dishwasher, No Fee,...
124009    [Dining Room, Elevator, Laundry in Building, D...
Name: features, Length: 49352, dtype: object

In [9]:
empty_count = df_copy['features'].apply(lambda x: len(x) == 0).sum() # Кол-во пустых значений
print(empty_count)

3218


### 3.2 Удалите неиспользуемые символы ([,],’,” и пробел) из столбца.

- Таких символов нет в данных

### 3.3 Разделите значения в каждой строке с помощью разделителя «,”» и соберите результат в один большой список для всего набора данных. Вы можете использовать DataFrame.iterrows().
### 3.4 Сколько уникальных значений содержится в списке результатов?

In [10]:
all_features = [] # Список характеристик

for index, row in df_copy.iterrows(): # Итерация по df_copy
    all_features.extend(row['features'])

len(set(all_features)) # Вывод уникальных значений

1556

### 3.5 Давайте познакомимся с новой библиотекой — Collections. С помощью этого пакета вы сможете эффективно получать количественную статистику по вашим данным.
### 3.6 Подсчитайте самые популярные функции из нашего огромного списка и выберите 20 лучших на данный момент.

### 3.7 Если все правильно, вы должны получить следующие значения: 'Elevator', 'Hardwoodf_copyloors', 'CatsAllowed', 'DogsAllowed', 'Doorman', 'Dishwasher', 'NoFee', 'LaundryinBuilding', 'FitnessCenter', 'Pre-War', 'LaundryinUnit', 'RoofDeck', 'OutdoorSpace', 'DiningRoom', 'HighSpeedInternet', 'Balcony', 'SwimmingPool', 'LaundryInBuilding', 'NewConstruction', 'Terrace'

In [11]:
top_features = Counter(all_features).most_common(20)
top_features

[('Elevator', 25915),
 ('Cats Allowed', 23540),
 ('Hardwood Floors', 23527),
 ('Dogs Allowed', 22035),
 ('Doorman', 20898),
 ('Dishwasher', 20426),
 ('No Fee', 18062),
 ('Laundry in Building', 16344),
 ('Fitness Center', 13252),
 ('Pre-War', 9148),
 ('Laundry in Unit', 8738),
 ('Roof Deck', 6542),
 ('Outdoor Space', 5268),
 ('Dining Room', 5136),
 ('High Speed Internet', 4299),
 ('Balcony', 2992),
 ('Swimming Pool', 2730),
 ('Laundry In Building', 2593),
 ('New Construction', 2559),
 ('Terrace', 2283)]

### 3.8 Теперь сгенерируйте 20 новых признаков на основе 20 лучших значений: 1, если значение в столбце "feature", иначе 0.

In [12]:
df = pd.DataFrame() # Создадим новый датафрейм

In [13]:
for feature, count in top_features:
    df[feature] = df_copy['features'].apply(lambda x: 1 if feature in x else 0) 

In [14]:
df.head()

,Elevator,Cats Allowed,Hardwood Floors,Dogs Allowed,Doorman,Dishwasher,No Fee,Laundry in Building,Fitness Center,Pre-War,Laundry in Unit,Roof Deck,Outdoor Space,Dining Room,High Speed Internet,Balcony,Swimming Pool,Laundry In Building,New Construction,Terrace
4,0,1,1,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0
6,1,0,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
9,1,0,1,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0


### 3.9 Расширим набор функций, добавив 'bathrooms',  'bedrooms', 'interest_level' и создадим специальную переменную feature_list со всеми названиями функций. Теперь у нас 23 значения. Все модели должны быть обучены на этих 23 функциях.

In [15]:
df = pd.concat([df, df_copy[['bathrooms',  'bedrooms', 'interest_level', 'price']]], axis=1)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49352 entries, 4 to 124009
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Elevator             49352 non-null  int64  
 1   Cats Allowed         49352 non-null  int64  
 2   Hardwood Floors      49352 non-null  int64  
 3   Dogs Allowed         49352 non-null  int64  
 4   Doorman              49352 non-null  int64  
 5   Dishwasher           49352 non-null  int64  
 6   No Fee               49352 non-null  int64  
 7   Laundry in Building  49352 non-null  int64  
 8   Fitness Center       49352 non-null  int64  
 9   Pre-War              49352 non-null  int64  
 10  Laundry in Unit      49352 non-null  int64  
 11  Roof Deck            49352 non-null  int64  
 12  Outdoor Space        49352 non-null  int64  
 13  Dining Room          49352 non-null  int64  
 14  High Speed Internet  49352 non-null  int64  
 15  Balcony              49352 non-null  int

In [17]:
df

,Elevator,Cats Allowed,Hardwood Floors,Dogs Allowed,Doorman,Dishwasher,No Fee,Laundry in Building,Fitness Center,Pre-War,...,High Speed Internet,Balcony,Swimming Pool,Laundry In Building,New Construction,Terrace,bathrooms,bedrooms,interest_level,price
4,0,1,1,1,0,1,0,1,0,1,...,0,0,0,0,0,0,1.0,1,1,2400
6,1,0,1,0,1,1,1,1,0,0,...,0,0,0,0,0,0,1.0,2,0,3800
9,1,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,1.0,2,1,3495
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1.5,3,1,3000
15,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,1.0,0,0,2795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124000,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1.0,3,0,2800
124002,1,1,0,1,1,0,1,0,0,0,...,0,0,0,1,0,0,1.0,2,1,2395
124004,1,1,1,1,0,1,1,1,0,1,...,0,0,0,0,0,0,1.0,1,1,1850
124008,0,0,0,0,0,1,1,0,0,1,...,0,0,0,0,0,0,1.0,2,1,4195


In [18]:
feature_list = df.columns.to_list()
feature_list

['Elevator',
 'Cats Allowed',
 'Hardwood Floors',
 'Dogs Allowed',
 'Doorman',
 'Dishwasher',
 'No Fee',
 'Laundry in Building',
 'Fitness Center',
 'Pre-War',
 'Laundry in Unit',
 'Roof Deck',
 'Outdoor Space',
 'Dining Room',
 'High Speed Internet',
 'Balcony',
 'Swimming Pool',
 'Laundry In Building',
 'New Construction',
 'Terrace',
 'bathrooms',
 'bedrooms',
 'interest_level',
 'price']

# 4. Реализация моделей - линейная регрессия

In [19]:
X = df.drop('price', axis=1)
y = df['price'] 

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

### 4.1 Реализуйте класс Python для алгоритма линейной регрессии с двумя базовыми методами — подгонки и прогнозирования. Используйте стохастический градиентный спуск для поиска оптимальных весов модели. Для лучшего понимания мы рекомендуем дополнительно реализовать отдельные версии алгоритма с аналитическим решением и нестохастическим градиентным спуском. 

#### Аналитическое решение

In [21]:
class LinearRegressionAnalytical:
    def __init__(self):
        self.weights = None # Массив для хранения весов модели

    '''Добавление столбеца с единицами (константа)'''
    def _add_intercept(self, X):
        return np.c_[np.ones((len(X), 1)), X]

    def fit(self, X, y):
        X_1 = self._add_intercept(X) # Создаем столбец из одних единиц, для учета свободного члена (константы).Точка пересечения с осью Y.
        self.weights = np.linalg.inv(X_1.T @ X_1) @ (X_1.T @ y)

    def predict(self, X):
        X_1 = self._add_intercept(X)
        return X_1 @ self.weights

In [22]:
lra = LinearRegressionAnalytical()

In [23]:
lra.fit(X_train, y_train)
lra.predict(X_train)

array([2539.02815679, 2277.55080871, 3627.07271382, ..., 1951.6763285 ,
       3174.84386718, 3715.77703646])

#### Градиентный спуск

In [24]:
class BatchGradientDescent:
    def __init__(self):
        self.weights = None # Массив для хранения весов модели
        self.eta = 0.01 # Шаг обучения (насколько изменить веса)
        self.n_iter = 200 # Кол-во итераций (сколько раз обновлять веса)
    
    '''Добавление столбеца с единицами (константа)'''
    def _add_intercept(self, X):
        return np.c_[np.ones((len(X), 1)), X]

    '''Предсказания на основе текущих весов модели'''
    def predict(self, X):
        X_1 = self._add_intercept(X)
        return X_1 @ self.weights
    
    def fit(self, X, y):
        X_1 = self._add_intercept(X)
        self.weights = np.zeros(X_1.shape[1]) # Инициализация весов нулями
        for iter in range(self.n_iter):
            predictions = X_1 @ self.weights # Предсказание на текущем шаге
            errors = predictions - y # Вычисляем ошибку
            gradient = (2 / len(y)) * X_1.T @ errors  # Вычисление градиента
            self.weights -= self.eta * gradient  # Обновление весов
            

In [25]:
bgd = BatchGradientDescent()
bgd.fit(X_train, y_train)
bgd.predict(X_train)

array([2397.42976593, 2421.08301767, 2991.6728728 , ..., 2021.69859141,
       3190.55208343, 3443.13928673])

#### Стохастический градиентный спуск

In [26]:
class StochasticGradientDescent:
    def __init__(self, n_epoch=10):
        self.weights = None # Массив для хранения весов модели
        self.n_epoch = n_epoch # Кол-во итераций (сколько раз обновлять веса)
        self.t0, self.t1 = 0.01, 1 # График обучения - определения скорости обучения
        self.m = len(y)

    '''Добавление столбеца с единицами (константа)'''
    def _add_intercept(self, X):
        return np.c_[np.ones((len(X), 1)), X]

    '''Предсказания на основе текущих весов модели'''
    def predict(self, X):
        X_1 = self._add_intercept(X)
        return X_1 @ self.weights
    
    '''График обучения'''
    def learning_schedule(self, t):
        return self.t0 / (t + self.t1)
    
    def fit(self, X, y):
        X_1 = self._add_intercept(X)
        self.weights = np.zeros(X_1.shape[1]) # Инициализация весов нулями
        for epoch in range(self.n_epoch):
            for iter in range(self.m):
                random_index = np.random.randint(self.m)
                xi = X_1[random_index : random_index + 1] # Случайны образец
                yi = y[random_index : random_index + 1]

                predictions = xi @ self.weights # Предсказание на текущем шаге
                errors = predictions - yi # Вычисляем ошибку

                gradient = 2 * xi.T @ errors  # Вычисление градиента

                eta = self.learning_schedule(epoch * self.m + iter) # Вычисление шага обучения в процессе обучения модели
                self.weights -= eta * gradient  # Обновление весов

In [27]:
sgd = StochasticGradientDescent()
sgd.fit(X_train, y_train)
sgd_train = sgd.predict(X_train)

### 4.2 Дайте определение для коэффициента R в квадрате (R2) и реализуйте функцию для вычисления.

**$R^2$ (коэффициента детерминации)** - измеряет, какая доля дисперсии одной переменной объяснятся дисперсией другой переменной.

Коэффициент детерминации $R^2$ используется для оценки качества регрессионной модели, показывая, насколько хорошо модель объясняет вариацию зависимой переменной.

Формула:

- $R^2 = 1 - \frac{SS_{res}}{SS_{tot}}$
  - $SS_{res}$ - сумма квадратов остатков (разница между фактическими и предсказанными значениями),
  - $SS_{tot}$ - общая сумма квадратов (разница между фактическими значениями и их средним).

In [28]:
'''Функция для вычисления R2'''
def r_squared(y_true, y_pred):
    y_true_mean = sum(y_true) / len(y_true) # Средние значение для истинных значений
    ss_tot = sum((y_true - y_true_mean) ** 2) # Сумма квадратов общего (SS_tot)
    ss_res = sum((y_true - y_pred) ** 2) # Сумма квадратов остатков (SS_res)
    r2 = 1 - (ss_res / ss_tot)
    return r2

### 4.3 Делайте прогнозы с помощью своего алгоритма и оценивайте модель с помощью показателей MAE, RMSE и R2. 

In [29]:
def metrics_mae_rmse_r2(y_true, y_pred):
    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(root_mean_squared_error(y_true, y_pred), 2)
    r2 = round(r2_score(y_true, y_pred), 2)
    return mae, rmse, r2

In [30]:
sgd_train_metrics = metrics_mae_rmse_r2(y_train, sgd_train)
sgd_train_metrics

(np.float64(1183.14), np.float64(9812.8), 0.01)

In [31]:
sgd_test = sgd.predict(X_test)
sgd_test_metrics = metrics_mae_rmse_r2(y_test, sgd_test)
sgd_test_metrics

(np.float64(1557.64), np.float64(45221.87), 0.0)

### 4.4 Инициализируйте LinearRegression() из sklearn.linear_model, подберите модель и прогнозируйте обучающие и тестовые части, так же, как в предыдущем уроке. 

In [32]:
model_ls = LinearRegression().fit(X_train, y_train)
model_ls_train = model_ls.predict(X_train)
model_ls_test = model_ls.predict(X_test)

In [33]:
model_lr_train_metrics = metrics_mae_rmse_r2(y_train, model_ls_train)
model_lr_train_metrics

(np.float64(973.05), np.float64(9708.55), 0.03)

In [34]:
model_lr_test_metrics = metrics_mae_rmse_r2(y_test, model_ls_test)
model_lr_test_metrics

(np.float64(1345.97), np.float64(45184.53), 0.0)

### 4.5 Сравните показатели качества и убедитесь, что разница невелика. (Между вашими реализациями и sklearn). 

In [35]:
'''Сравнение показателей качества train (Между нашими реализациями и sklear)'''
results_train = tuple(round(abs(a - b), 2) for a, b in zip(sgd_train_metrics, model_lr_train_metrics))
print(results_train)

(np.float64(210.09), np.float64(104.25), 0.02)


In [36]:
'''Сравнение показателей качества test (Между нашими реализациями и sklear)'''
results_test = tuple(round(a - b, 2) for a, b in zip(sgd_test_metrics, model_lr_test_metrics))
print(results_test)

(np.float64(211.67), np.float64(37.34), 0.0)


### 4.6 Сохраните показатели, как в предыдущем уроке, в таблице со столбцами model, train, test for MAE table, RMSE table и R2 coefficient.

In [37]:
'''Создание DataFrame и добавление данных'''
def append_row_table(df_name, model, train, test):
    # Проверяем, существует ли DataFrame
    if df_name not in globals():
        # Если не существует, создаем новый DataFrame
        globals()[df_name] = pd.DataFrame(columns=['model', 'train', 'test'])
    
    # Добавляем новую строку в DataFrame
    globals()[df_name].loc[len(globals()[df_name])] = [model, train, test]

In [38]:
append_row_table('df_mae', 'StochasticGradientDescent', sgd_train_metrics[0], sgd_test_metrics[0])
append_row_table('df_rmse', 'StochasticGradientDescent', sgd_train_metrics[1], sgd_test_metrics[1])
append_row_table('df_r2', 'StochasticGradientDescent', sgd_train_metrics[2], sgd_test_metrics[2])

In [39]:
df_mae

,model,train,test
0,StochasticGradientDescent,1183.14,1557.64


In [40]:
df_rmse

,model,train,test
0,StochasticGradientDescent,9812.8,45221.87


In [41]:
df_r2

,model,train,test
0,StochasticGradientDescent,0.01,0.0


## 5. Реализация регуляризованных моделей - Гребень, Лассо, Эластичная сетка

### 5.1 Реализуйте алгоритмы Ridge, Lasso, ElasticNet: расширьте функцию потерь с помощью L2, L1 и обеих регуляризаций соответственно. 

#### Ridge

In [68]:
class Ridge_Reg:
    def __init__(self, alpha=1.0):
        self.weights = None # Массив для хранения весов модели
        self.alpha = alpha # Гиперпараметр коэффициент регуляризации

    '''Добавление столбеца с единицами (константа)'''
    def _add_intercept(self, X):
        return np.c_[np.ones((len(X), 1)), X]

    def fit(self, X, y):
        n_features = X.shape[1]
        I = np.eye(n_features + 1)
        X_1 = self._add_intercept(X) # Создаем столбец из одних единиц, для учета свободного члена (константы).Точка пересечения с осью Y.
        self.weights = np.linalg.inv(X_1.T @ X_1 + self.alpha * I) @ (X_1.T @ y)
        return self

    def predict(self, X):
        X_1 = self._add_intercept(X)
        return X_1 @ self.weights

In [69]:
ridge_reg = Ridge_Reg().fit(X_train, y_train)
ridge_reg.predict(X_train)

array([2538.98798194, 2277.58863152, 3626.83385073, ..., 1951.74812272,
       3174.816225  , 3715.6765507 ])

#### Lasso

In [ ]:
class Lasso_Reg:
    def __init__(self, alpha=1.0):
        self.weights = None # Массив для хранения весов модели
        self.alpha = alpha # Гиперпараметр коэффициент регуляризации

    '''Добавление столбеца с единицами (константа)'''
    def _add_intercept(self, X):
        return np.c_[np.ones((len(X), 1)), X]

    def fit(self, X, y):
        n_features = X.shape[1]
        I = np.eye(n_features + 1)
        X_1 = self._add_intercept(X) # Создаем столбец из одних единиц, для учета свободного члена (константы).Точка пересечения с осью Y.
        self.weights = np.linalg.inv(X_1.T @ X_1 + self.alpha * I) @ (X_1.T @ y)
        return self

    def predict(self, X):
        X_1 = self._add_intercept(X)
        return X_1 @ self.weights